test

In [1]:
import os
import sagemaker
from sagemaker import get_execution_role

sagemaker_session = sagemaker.Session()

#role = get_execution_role()
role = "arn:aws:iam::941656036254:role/service-role/AmazonSageMaker-ExecutionRole-20210904T193230"
region = sagemaker_session.boto_session.region_name

In [4]:
training_data_uri = 's3://sagemaker-sample-data-{}/tensorflow/mnist'.format(region)


In [7]:
from sagemaker.tensorflow import TensorFlow

mnist_estimator = TensorFlow(entry_point='mnist.py',
                             source_dir="3_src",
                             role=role,
                             instance_count=1,
                             instance_type='ml.p3.2xlarge',
                             framework_version='1.15.0',
                             py_version='py3',
                            # distribution={'parameter_server': {'enabled': True}}
                             )

In [8]:
mnist_estimator.fit(training_data_uri)


2022-07-07 16:26:45 Starting - Starting the training job...ProfilerReport-1657211205: InProgress
...
2022-07-07 16:27:29 Starting - Preparing the instances for training......
2022-07-07 16:28:47 Downloading - Downloading input data
2022-07-07 16:28:47 Training - Downloading the training image.........
2022-07-07 16:30:10 Training - Training image download completed. Training in progress.WARNING:tensorflow:From /usr/local/lib/python3.6/dist-packages/tensorflow_core/__init__.py:1467: The name tf.estimator.inputs is deprecated. Please use tf.compat.v1.estimator.inputs instead.
2022-07-07 16:30:04,196 sagemaker-containers INFO     Imported framework sagemaker_tensorflow_container.training
2022-07-07 16:30:04,794 sagemaker-containers INFO     Invoking user script
Training Env:
{
    "additional_framework_parameters": {},
    "channel_input_dirs": {
        "training": "/opt/ml/input/data/training"
    },
    "current_host": "algo-1",
    "framework_module": "sagemaker_tensorflow_container.t

## Compile for Inferentia

In [9]:
output_path = '/'.join(mnist_estimator.output_path.split('/')[:-1])

inf_estimator = mnist_estimator.compile_model(target_instance_family='ml_inf1', 
                              input_shape={'data':[1, 784]},  # Batch size 1, 1 channels, 28x28 Images.
                              output_path=output_path)

???????????????????????????????.................................................!

In [10]:
inf_predictor = inf_estimator.deploy(initial_instance_count = 1,
                                                 instance_type = 'ml.inf1.xlarge')

update_endpoint is a no-op in sagemaker>=2.
See: https://sagemaker.readthedocs.io/en/stable/v2.html for details.


-----------!

## Compile for CPU instance

In [11]:
c5_estimator = mnist_estimator.compile_model(target_instance_family='ml_c5', 
                              input_shape={'data':[1, 784]},  # Batch size 1, 1 channels, 28x28 Images.
                              output_path=output_path)

??????????????????????????.....................................!

In [12]:
c5_predictor = c5_estimator.deploy(initial_instance_count = 1,
                                                 instance_type = 'ml.c5.xlarge')

update_endpoint is a no-op in sagemaker>=2.
See: https://sagemaker.readthedocs.io/en/stable/v2.html for details.


----!

## Testing inference

In [19]:
! aws s3 cp s3://sagemaker-sample-data-us-east-1/tensorflow/mnist . --recursive

download: s3://sagemaker-sample-data-us-east-1/tensorflow/mnist/eval_labels.npy to ./eval_labels.npy
download: s3://sagemaker-sample-data-us-east-1/tensorflow/mnist/train_labels.npy to ./train_labels.npy
download: s3://sagemaker-sample-data-us-east-1/tensorflow/mnist/eval_data.npy to ./eval_data.npy
download: s3://sagemaker-sample-data-us-east-1/tensorflow/mnist/train_data.npy to ./train_data.npy


In [20]:
import numpy as np

inference_data = np.load("eval_data.npy")
inference_labels = np.load("eval_labels.npy")

In [41]:
#print(inference_data[0].reshape(28,28)*255)
print(inference_data[0].reshape(28,28).shape)
print(inference_data[0:2].reshape(2,784).shape)

(28, 28)
(2, 784)


In [71]:
#inf_predictor.predict(inference_data[0:4].reshape(4,784))
c5_predictor.predict(inference_data[0].reshape(1,784))

{'predictions': [{'probabilities': [1.18185801e-06,
    2.26060507e-07,
    2.78294756e-05,
    7.85213924e-05,
    1.07752491e-07,
    1.51988343e-06,
    4.83612927e-10,
    0.999833465,
    1.04196897e-06,
    5.59712244e-05],
   'classes': 7}]}

In [73]:
for i in range(10):
    data = inference_data[i].reshape(1,784)
    # Follow https://www.tensorflow.org/tfx/serving/api_rest guide to format input to the model server
    predict_response = c5_predictor.predict({"instances": np.asarray(data).tolist()})

    print("========================================")
    label = np.argmax(inference_labels[i])
    print("label is {}".format(label))
    prediction = np.argmax(predict_response["predictions"])
    print("prediction is {}".format(prediction))

0
label is 0
prediction is 0
1
label is 0
prediction is 0
2
label is 0
prediction is 0
3
label is 0
prediction is 0
4
label is 0
prediction is 0
5
label is 0
prediction is 0
6
label is 0
prediction is 0
7
label is 0
prediction is 0
8
label is 0
prediction is 0
9
label is 0
prediction is 0


In [62]:
import tensorflow_datasets
mnist_test = tensorflow_datasets.load('mnist', split=[ 'test'],
    shuffle_files=True,
    as_supervised=True,
    with_info=True)

In [50]:
for i in range(10):
    data = mnist.test.images[i].tolist()
    # Follow https://www.tensorflow.org/tfx/serving/api_rest guide to format input to the model server
    predict_response = inf_predictor.predict({"instances": np.asarray(data).tolist()})

    print("========================================")
    label = np.argmax(mnist.test.labels[i])
    print("label is {}".format(label))
    prediction = np.argmax(predict_response["predictions"])
    print("prediction is {}".format(prediction))

AttributeError: 'dict' object has no attribute 'test'

In [69]:
mnist_test[0][0]['label']

TypeError: 'PrefetchDataset' object is not subscriptable